<a href="https://colab.research.google.com/github/RaianaSeixas/Paper-1-Final-Results/blob/master/Stats_2000_Rosenbrock_ALL3_D_30_0_75_0_75.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install pyade-python

In [24]:
!pip install PyFDE

In [25]:
import pyade.ilshade 
import pyade.sade 
import pyade.jso
import pyade.shade
import pyade.jade 
import pyfde
import numpy as np
import pandas as pd
import time

In [26]:
!git clone https://github.com/ucfilho/Raianars_July_2020 #clonar do Github
%cd Raianars_July_2020

Cloning into 'Raianars_July_2020'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 138 (delta 7), reused 0 (delta 0), pack-reused 123
Receiving objects: 100% (138/138), 201.58 KiB | 7.75 MiB/s, done.
Resolving deltas: 100% (69/69), done.
/content/Raianars_July_2020/ANN_read_matrix/Raianars_July_2020/Raianars_July_2020/Raianars_July_2020


In [27]:
import Go2Ann
import de_soma_Insight
import de_bounds_matriz
from AvaliaX import AvaliaX
from GeraIndices import GeraIndices

In [28]:
import Function
import Go2MinMax

In [29]:
ITE=2000 #10 # PONTOS quantas vezes vai alteral NPAR 
NPAR=100 # NUMBER OF PARTICLES
dim=30
RUNS=50
fchoice='Rosenbrock'  
Fun=Function.Rosenbrock

MIN,MAX=Go2MinMax.Intervalo(dim,fchoice)
bounds = [(0,0)]*dim

for i in range(dim):
  bounds[i] = (MIN[i],MAX[i])

start_time = time.time() 

In [30]:
############################### INPUT INFORMATION (FRANNK)
Fc=0.1 # fator de correcao parametros da ANN
its=1 #10 # quantas vezes vai usar o NPAR antes de alterar de novo
mut=0.75
crossp=0.75
Fi=mut;CRi=crossp;# para impressão
PAR=len(MAX) #NUM DE PARAMETROS A SER OTIMIZADOS

In [31]:
# JADE
algorithm = pyade.jso
BEST_AGENTS=[]
for i in range(RUNS):
  ANSWER_AGENTS=[]
  solver = pyfde.JADE(Fun, n_dim=dim, n_pop=NPAR, limits=bounds, minimize=True)
  solver.p = 0.05
  solver.c = 0.1
  solver.cr = 0.5
  solver.f = 0.5
  best, fit = solver.run(n_it=ITE)
  for x in best:
    ANSWER_AGENTS.append(x)
  ANSWER_AGENTS.append(fit)
  BEST_AGENTS.append(ANSWER_AGENTS)
BEST_AGENTS=np.array(BEST_AGENTS)
TIME = (time.time() - start_time)/RUNS
BEST_AGENTS=np.array(BEST_AGENTS)
BEST_AGENTS_ST= BEST_AGENTS[BEST_AGENTS[:,-1].argsort()]
BEST=BEST_AGENTS_ST[0]
WORSE=BEST_AGENTS_ST[-1]
MEAN=np.mean(BEST_AGENTS,axis=0)
MAXFO=BEST_AGENTS[:,-1].max()
MINFO=BEST_AGENTS[:,-1].min()
STD=BEST_AGENTS[:,-1].std()
nome='JADE_Fo-'+str(0)+'_CRo-'+str(0)
print(nome,'NPAR',NPAR,'Iterações=',ITE,'Dim=',dim ,"Min/Max=",MIN[0],"/",MAX[0],'_',Fun.__name__)
print( '\n')
print('MÉDIA: ',MEAN[-1])
print('MIN __:',MINFO)
print('MAX__: ',MAXFO)
print('STD __:',STD)
print("Tempo Médio= %.3f segundos ---" % (TIME))
print(RUNS,'Rodadas') 
print()
print('BEST: ',BEST)

JADE_Fo-0_CRo-0 NPAR 100 Iterações= 2000 Dim= 30 Min/Max= -10 / 10 _ Rosenbrock


MÉDIA:  -0.07973247708601859
MIN __: -3.98662385430093
MAX__:  -1.055166751298962e-24
STD __: 0.5581273396021301
Tempo Médio= 5.329 segundos ---
50 Rodadas

BEST:  [-0.9932861   0.99665107  0.99833032  0.99916774  0.9995852   0.99979328
  0.99989698  0.99994866  0.99997442  0.99998725  0.99999365  0.99999683
  0.99999842  0.99999921  0.99999961  0.9999998   0.9999999   0.99999995
  0.99999998  0.99999999  0.99999999  1.          1.          1.
  1.          1.          1.          1.          0.99999999  0.99999998
 -3.98662385]


In [32]:
# FRANNK START HERE !!!!!

In [33]:
!git clone https://github.com/ucfilho/ANN_read_matrix #clonar do Github
%cd ANN_read_matrix

Cloning into 'ANN_read_matrix'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 20 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (20/20), done.
/content/Raianars_July_2020/ANN_read_matrix/Raianars_July_2020/Raianars_July_2020/Raianars_July_2020/ANN_read_matrix


In [34]:
ANN_F=pd.read_csv('ANN_INSIGHT_Valor_F.csv')
ANN_CR=pd.read_csv('ANN_INSIGHT_CR.csv')
syn0_F,syn1_F,X_max_F,X_min_F=Go2Ann.GetMatriz(ANN_F)
syn0_CR,syn1_CR,X_max_CR,X_min_CR=Go2Ann.GetMatriz(ANN_CR)
fields = ['DI RELATIVO','FRAC Its','Fo','CRo']

#TOTAL=7*dim*its*ITE
TOTAL=10*dim*its*ITE
TOTAL=int(TOTAL/2)   # CONFERIR SE ESTE EH O MELHOR LUGAR

dimensions=len(MAX)
start_time = time.time() #INICIALIZA CONTAGEM DO TEMPO

MAX_P=np.zeros(ITE) # ATENCAO para variar F e Cr MAX_P passa a ser 3 vezes maior
MIN_P=np.zeros(ITE) # ATENCAO para variar F e Cr MIN_P passa a ser 3 vezes maior

for i in range(dim):
  MAX_P[i]=MAX[i] 
  MIN_P[i]=MIN[i]

rodadas=1
for i in range(rodadas):
  SOMA=0
  pop = np.random.rand(NPAR, dimensions)
  diff=np.zeros(PAR)
  for ii in range(PAR):
    diff[ii] = MAX[ii]-MIN[ii]
  X = MIN + pop * diff
    
  PONTOS=ITE # DIVISOES DA POPULACAO
start_time = time.time()
BEST_AGENTS=[]
MAT_stat=[]
for i in range(RUNS):
    MAT=[] 
    mut=Fi
    crossp=CRi
    SOMA=0
    SOMAf=0
    start_ite = time.time()
    delta_time=(start_ite-start_time)/60
    print('ite=',i,'time=',delta_time)
    pop = np.random.rand(NPAR, dim)
    diff=np.zeros(PAR)
    for ii in range(PAR):
      diff[ii] = MAX[ii]-MIN[ii]
    X = MIN + pop * diff
    Xf = MIN + pop * diff
    
    DIo=1e99
  
    CRo=crossp;Fo=mut;
    Fitness = np.asarray([Fun(ind) for ind in X])

    XY,BEST_XY,BESTo,FOBESTo=AvaliaX(X,Fitness)
    nrow,ncol=X.shape

    QUANT=17;MAT_INDo=np.zeros((1,QUANT)); # quantos indices esta fazendo
    MAT_INDo[0,5]=nrow  # VALOR NP
    MAT_INDo[0,6]=Fo    # VALOR Fo   
    MAT_INDo[0,7]=CRo   # VALOR CRo
  
    MAT_INDo=GeraIndices(X,BESTo,FOBESTo,DIo,MAT_INDo,SOMA,TOTAL,syn0_F,
                        syn1_F,X_max_F,X_min_F,syn0_CR,syn1_CR,X_max_CR,
                        X_min_CR,Fc,fields,Fun)
  
    DIo=np.copy(MAT_INDo[0,0])
    MAT_INDo[0,15]=Fo    # VALOR Fo   
    MAT_INDo[0,16]=CRo   # VALOR CRo
    #ind_crossp=ITE-1

    
    for k in range(ITE):
      X,BEST,FOBEST,XY,BEST_XY,SOMA= de_soma_Insight.de(MAX,MIN, mut, crossp,NPAR,its,Fun,X,SOMA,TOTAL)
      #Xf,BESTf,FOBESTf,XYf,BEST_XYf,SOMAf= de_soma_Insight.de(MAX,MIN, Fi, CRi, NPAR,its,Fun,Xf,SOMAf,TOTAL)

      
      MAT_IND=GeraIndices(X,BESTo,FOBESTo,DIo,MAT_INDo,SOMA,TOTAL,syn0_F,
                          syn1_F,X_max_F,X_min_F,syn0_CR,syn1_CR,X_max_CR,
                          X_min_CR,Fc,fields,Fun)
    
      BESTo=np.copy(BEST);FOBESTo=np.copy(FOBEST);
      MAT.append(MAT_IND.ravel())
    
      MAT_INDo[0,6]=MAT_IND[0,15]   # VALOR Fo   
      MAT_INDo[0,7]=MAT_IND[0,16]   # VALOR CRo
      mut=MAT_IND[0,15]
      crossp=MAT_IND[0,16]

      Fitness = np.asarray([Fun(ind) for ind in X])
      #Fitnessf = np.asarray([Fun(ind) for ind in Xf])
      Maior=np.argmax(Fitness)
      #Maiorf=np.argmax(Fitnessf)
      Menor=np.argmin(Fitness)
      #Menorf=np.argmin(Fitnessf)

      #X[Maior,:]=np.copy(Xf[Menorf,:])
      #Xf[Maiorf,:]=np.copy(X[Menor,:])
      
    BEST_AGENTS.append(BEST_XY)
    MAT_stat.append(MAT)
    #print("GlobalParam e Solucao:", BEST,"Funcao=",FOBEST)
    
############################## Cálculo dos Parâmetros de Avaliação   
TIME = (time.time() - start_time)/RUNS
BEST_AGENTS=np.array(BEST_AGENTS)
BEST_AGENTS_ST= BEST_AGENTS[BEST_AGENTS[:,-1].argsort()]
BEST=BEST_AGENTS_ST[0]
WORSE=BEST_AGENTS_ST[-1]
MEAN=np.mean(BEST_AGENTS,axis=0)
MAXFO=BEST_AGENTS[:,-1].max()
MINFO=BEST_AGENTS[:,-1].min()
STD=BEST_AGENTS[:,-1].std()

print()
nome='_ANN_Div3x4_Fo-'+str(Fi)+'_CRo-'+str(CRi)+'_D-'+str(dim)+'_Interv-'+str(ITE)
print('DE_POP_VAR_F_CR','TOTAL=',TOTAL,'Iterações=',ITE*its,'Dim=', PAR,"Min/Max=",MIN[0],"/",MAX[0],'_',Fun.__name__)
print(nome, '\n')
print('MÉDIA: ',MEAN[-1])
print('MIN __:',MINFO)
print('MAX__: ',MAXFO)
print('STD __:',STD)
print("Tempo Médio= %.3f segundos ---" % (TIME))
print(RUNS,'Rodadas') 
print()
print('BEST: ',BEST)

ite= 0 time= 2.509752909342448e-05
ite= 1 time= 3.017078502972921
ite= 2 time= 6.022419528166453
ite= 3 time= 9.035946317513783
ite= 4 time= 12.044674722353617
ite= 5 time= 15.058904739220937
ite= 6 time= 18.07715051571528
ite= 7 time= 21.099872799714408
ite= 8 time= 24.10721247990926
ite= 9 time= 27.11824411948522
ite= 10 time= 30.128573489189147
ite= 11 time= 33.13190060853958
ite= 12 time= 36.13570831219355
ite= 13 time= 39.14671000242233
ite= 14 time= 42.15052237113317
ite= 15 time= 45.158575518925986
ite= 16 time= 48.16979146798452
ite= 17 time= 51.17775307099024
ite= 18 time= 54.18911195596059
ite= 19 time= 57.19633116324743
ite= 20 time= 60.20716214974721
ite= 21 time= 63.221540892124175
ite= 22 time= 66.22716236511866
ite= 23 time= 69.24121053218842
ite= 24 time= 72.25469896793365
ite= 25 time= 75.26498067378998
ite= 26 time= 78.27722640434901
ite= 27 time= 81.29277667204539
ite= 28 time= 84.30183914105098
ite= 29 time= 87.32263786395391
ite= 30 time= 90.34063846667608
ite= 31 

In [35]:
#DE_ tradicional

!git clone https://github.com/ucfilho/Raianars_July_2020 #clonar do Github
%cd Raianars_July_2020

mut=0.5
crossp=0.5

SOMA=0
start_time = time.time()
BEST_AGENTS=[]
MAT_stat=[]

for i in range(RUNS):

    pop = np.random.rand(NPAR, dim)
    diff=np.zeros(dim)
    for ii in range(dim):
      diff[ii] = MAX[ii]-MIN[ii]
    X = MIN + pop * diff

    SOMA=0

    for k in range(ITE):
      X,BEST,FOBEST,XY,BEST_XY,SOMA= de_soma_Insight.de(MAX,MIN, mut, crossp,dim,its,Fun,X,SOMA,TOTAL)

    start_run = (time.time()-start_time)/60
    print('run=',i,'total=50','Fobj=',FOBEST,'Time=',start_run)  
    BEST_AGENTS.append(BEST_XY)

Cloning into 'Raianars_July_2020'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 138 (delta 7), reused 0 (delta 0), pack-reused 123
Receiving objects: 100% (138/138), 201.58 KiB | 6.11 MiB/s, done.
Resolving deltas: 100% (69/69), done.
/content/Raianars_July_2020/ANN_read_matrix/Raianars_July_2020/Raianars_July_2020/Raianars_July_2020/ANN_read_matrix/Raianars_July_2020
run= 0 total=50 Fobj= 25.329153609141095 Time= 0.4981286644935608
run= 1 total=50 Fobj= 25.108251824817422 Time= 0.995913028717041
run= 2 total=50 Fobj= 23.598944229475165 Time= 1.4924718777338664
run= 3 total=50 Fobj= 24.5136296803202 Time= 1.989747707049052
run= 4 total=50 Fobj= 25.477430060727123 Time= 2.4994517167409263
run= 5 total=50 Fobj= 24.504972018165677 Time= 2.9959537823994955
run= 6 total=50 Fobj= 25.693300826682837 Time= 3.4937187989552814
run= 7 total=50 Fobj= 24.914617970295243 Time= 3.9904160141944884


In [36]:
############################## Cálculo dos Parâmetros de Avaliação   
TIME = (time.time() - start_time)/RUNS
BEST_AGENTS=np.array(BEST_AGENTS)
BEST_AGENTS_ST= BEST_AGENTS[BEST_AGENTS[:,-1].argsort()]
BEST=BEST_AGENTS_ST[0]
WORSE=BEST_AGENTS_ST[-1]
MEAN=np.mean(BEST_AGENTS,axis=0)
MAXFO=BEST_AGENTS[:,-1].max()
MINFO=BEST_AGENTS[:,-1].min()
STD=BEST_AGENTS[:,-1].std()

nome='DE_STAT_Fo-'+str(mut)+'_CRo-'+str(crossp)+'_D-'+str(dim)+'_Interv-'+str(ITE)
print('DE_','TOTAL=',TOTAL,'Iterações=',ITE*its,'Dim=', dim,"Min/Max=",MIN[0],"/",MAX[0],'_',Fun.__name__)
print(nome, '\n')
#print('BEST: ',BEST)
#print('MÉDIA: ',MEAN)
print('MÉDIA: ',MEAN[-1])
print('MIN __:',MINFO)
print('MAX__: ',MAXFO)
print('STD __:',STD)
print("Tempo Médio= %.3f segundos ---" % (TIME))
print(RUNS,'Rodadas') 
print()
print('BEST: ',BEST)

DE_ TOTAL= 300000 Iterações= 2000 Dim= 30 Min/Max= -10 / 10 _ Rosenbrock
DE_STAT_Fo-0.5_CRo-0.5_D-30_Interv-2000 

MÉDIA:  24.696716663265683
MIN __: 19.843287582438638
MAX__:  25.693300826682837
STD __: 0.8997736738859742
Tempo Médio= 29.857 segundos ---
50 Rodadas

BEST:  [9.96652265e-01 9.93657181e-01 9.84831873e-01 9.72111207e-01
 9.46592698e-01 8.96317656e-01 8.11086035e-01 6.62289909e-01
 4.46605317e-01 2.09756335e-01 5.18728107e-02 1.12780690e-02
 9.07011431e-03 1.04924616e-02 1.15498897e-02 1.06242577e-02
 1.08946238e-02 1.22683334e-02 9.76384966e-03 8.41972936e-03
 7.36265152e-03 1.02700679e-02 7.51984804e-03 9.80062421e-03
 1.22808231e-02 1.08835200e-02 9.61769751e-03 9.12941271e-03
 1.12618521e-02 1.46705207e-03 1.98432876e+01]
